In [ ]:
import os
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from PIL import Image
from tqdm import tqdm

# Ensure PIL version compatibility (for resize)
try:
    # PIL 9.0.0+
    LANCZOS_RESAMPLE = Image.Resampling.LANCZOS
except AttributeError:
    # Older PIL
    LANCZOS_RESAMPLE = Image.LANCZOS

# --- 1. Define constants and paths ---
# Please modify the root path here according to your environment
DATA_ROOT = "/mnt/hdd/jiazy/petfinder-pawpularity-score"
METADATA_FILE = os.path.join(DATA_ROOT, "train.csv")
IMAGE_DIR = os.path.join(DATA_ROOT, "train")

# Output directory
OUTPUT_DIR = "./features"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# --- 2. Define feature columns ---
# Continuous features
CONTINUOUS_COLS = []

# Categorical features
CATEGORICAL_COLS = [
    'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage'
]

# Label column
LABEL_COL = 'Pawpularity'

# Non-feature columns (will be removed)
NON_FEATURE_COLS = [
    'Id', 
]

# --- 3. Image processing helper function ---
def process_and_save_image(png_path):
    """
    Load PNG image, resize to 224x224, and save as .npy file.
    If .npy file already exists, skip.
    """
    if not os.path.exists(png_path):
        print(f"Warning: Image file does not exist {png_path}")
        return None  # Return None for later filtering

    # Replace .png with .npy
    npy_path = png_path.replace(".jpg", ".npy")

    if os.path.exists(npy_path):
        return npy_path  # File already exists

    try:
        # Open image
        img = Image.open(png_path)
        
        # Resize to 224x224
        img_resized = img.resize((224, 224), resample=LANCZOS_RESAMPLE)
        
        # Convert to NumPy array
        if img_resized.mode == 'RGBA':
            img_resized = img_resized.convert('RGB')
            
        np_img = np.array(img_resized)
        
        # Save as .npy
        np.save(npy_path, np_img)
        
        return npy_path
    except Exception as e:
        print(f"Failed to process image {png_path}: {e}")
        return None

def preprocess_petfinder_flow(metadata_file, image_dir, output_dir, label_col):
    """
    Preprocess PetFinder dataset
    """
    print("Starting PetFinder dataset preprocessing pipeline...")
    os.makedirs(output_dir, exist_ok=True)
    
    # --- 1. 📖 Load data ---
    print(f"Loading data {metadata_file}...")
    df = pd.read_csv(metadata_file)
    print(f"Successfully loaded {len(df)} records.")

    print("Constructing image paths...")
    df['image_path'] = df['Id'].apply(lambda x: os.path.join(image_dir, f"{x}.jpg"))


    # --- 2. 📝 Define features ---
    print("Defining features...")
    continuous_cols = CONTINUOUS_COLS
    categorical_cols = CATEGORICAL_COLS
    non_feature_cols = NON_FEATURE_COLS
    exclude_cols = [label_col, 'image_path', 'npy_path'] + non_feature_cols
    
    # --- 3. 🧹 Data cleaning ---
    print("Cleaning data...")
    
    # Filter rows with missing image paths and labels
    df = df.dropna(subset=[label_col])
    print("Encoding categorical features...")
    cat_dims = [] # Record the number of categories for each feature
    for col in categorical_cols:
        df[col] = df[col].fillna(-1) # Simple filling
        df[col] = df[col].astype('category')
        df[col] = df[col].cat.codes
        # Record category count (max ID + 1)
        cat_dims.append(int(df[col].max() + 1))

    # --- 6. Dataset splitting ---
    print("Performing 80:10:10 dataset splitting...")
    # Regression tasks usually don't need stratify, or need bucketed stratify. For simplicity, use random splitting here.
    train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    
    print(f"Split results: Train {len(train_df)}, Validation {len(val_df)}, Test {len(test_df)}")

    # --- 7. Continuous feature standardization (if any) ---
    if continuous_cols:
        print("Standardizing continuous features...")
        scaler = StandardScaler()
        scaler.fit(train_df[continuous_cols])
        train_df[continuous_cols] = scaler.transform(train_df[continuous_cols])
        val_df[continuous_cols] = scaler.transform(val_df[continuous_cols])
        test_df[continuous_cols] = scaler.transform(test_df[continuous_cols])

    # --- 8. Image processing ---
    print("Processing images (Resize & Save .npy)...")
    tqdm.pandas(desc="Train Images")
    train_df['npy_path'] = train_df['image_path'].progress_apply(process_and_save_image)
    
    tqdm.pandas(desc="Val Images")
    val_df['npy_path'] = val_df['image_path'].progress_apply(process_and_save_image)
    
    tqdm.pandas(desc="Test Images")
    test_df['npy_path'] = test_df['image_path'].progress_apply(process_and_save_image)

    # Filter out failed image processing (those returning None)
    len_before = len(train_df) + len(val_df) + len(test_df)
    train_df = train_df.dropna(subset=['npy_path'])
    val_df = val_df.dropna(subset=['npy_path'])
    test_df = test_df.dropna(subset=['npy_path'])
    len_after = len(train_df) + len(val_df) + len(test_df)
    if len_before != len_after:
        print(f"Warning: Filtered {len_before - len_after} rows with missing or corrupted images.")

    # --- 9. Save output ---
    print(f"[Final] Saving processed files to {output_dir} ...")

    # Save field lengths: categorical first, then continuous (meets TIP model requirements)
    tabular_lengths = cat_dims + [1] * len(continuous_cols)
    torch.save(tabular_lengths, os.path.join(output_dir, "tabular_lengths.pt"))
    
    for split_name, df_split in zip(["train", "val", "test"], [train_df, val_df, test_df]):
        features_path = os.path.join(output_dir, f"{split_name}_features.csv")
        # Ensure column order: categorical first, then continuous
        cols_to_save = categorical_cols + continuous_cols
        df_split[cols_to_save].to_csv(features_path, index=False, header=False)
        
        labels_path = os.path.join(output_dir, f"{split_name}_labels.pt")
        # [Fixed] Convert labels to float32 for regression
        labels_tensor = torch.tensor(df_split[label_col].values, dtype=torch.float32)
        torch.save(labels_tensor, labels_path)
        
        paths_path = os.path.join(output_dir, f"{split_name}_paths.pt")
        # Save absolute paths of .npy files
        npy_path_list = df_split['npy_path'].tolist()
        torch.save(npy_path_list, paths_path)

    print("Preprocessing completed!")

if __name__ == "__main__":
    preprocess_petfinder_flow(
        metadata_file=METADATA_FILE,
        image_dir=IMAGE_DIR,
        output_dir=OUTPUT_DIR,
        label_col=LABEL_COL
    )

开始执行 PetFinder 数据集预处理流程...
正在加载数据 /mnt/hdd/jiazy/petfinder-pawpularity-score/train.csv...
成功加载 9912 条记录。
正在构造图像路径...
正在定义特征...
正在清洗数据...
正在进行类别特征编码...
正在进行 80:10:10 数据集划分...
划分结果: 训练集 7929, 验证集 991, 测试集 992
正在处理图像 (Resize & Save .npy)...


Test Images: 100%|██████████| 992/992 [00:31<00:00, 31.56it/s]


[Final] 正在保存处理后的文件到 ./features ...
预处理全部完成！
